In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import os
from collections import Counter

# ------------------- تنظیمات -------------------
base_path = r"C:\BI\KZCCPP\G11\Lube Oil System"
input_file = os.path.join(base_path, "lube_oil_system_data_g11.xlsx")
output_anomalies = os.path.join(base_path, "lube_oil_system_anomalies_g11.xlsx")
output_full_model = os.path.join(base_path, "dbscan_model1.xlsx")

MIN_CLUSTER_SIZE = 15  # خوشه‌های کوچکتر از این → انومالی
eps = 0.5
min_samples = 10

# ------------------- خواندن داده -------------------
print("در حال خواندن داده‌ها...")
df = pd.read_excel(input_file)

# ستون‌های سنسور
sensor_columns = [col for col in df.columns if col.startswith('AssetID_') and str(col.split('_')[-1]).isdigit()]
print(f"ستون‌های سنسور: {len(sensor_columns)} تا")

id_columns = ['id', 'RecordDate', 'RecordTime']
optional_cols = ['unitID', 'TimeStamps', 'created_at', 'updated_at']
available_optional = [col for col in optional_cols if col in df.columns]

# ------------------- پیش‌پردازش -------------------
X = df[sensor_columns].copy()
X = X.fillna(X.mean())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# ------------------- DBSCAN -------------------
print("اجرای DBSCAN...")
db = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)
labels = db.fit_predict(X_scaled)

# ------------------- تحلیل خوشه‌ها -------------------
label_counts = Counter(labels)
large_cluster_labels = [lbl for lbl, cnt in label_counts.items() if lbl != -1 and cnt >= MIN_CLUSTER_SIZE]
small_cluster_labels = [lbl for lbl, cnt in label_counts.items() if lbl != -1 and cnt < MIN_CLUSTER_SIZE]
noise_mask = labels == -1

# نقاط متعلق به خوشه‌های بزرگ (برای محاسبه فاصله)
large_cluster_mask = np.isin(labels, large_cluster_labels)
X_large = X_scaled[large_cluster_mask]

# ------------------- محاسبه فاصله تا نزدیک‌ترین خوشه اصلی -------------------
print("محاسبه شدت انومالی (model_value)...")
if len(X_large) > 0:
    nbrs = NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit(X_large)
    distances, _ = nbrs.kneighbors(X_scaled)
    distances = distances.flatten()
else:
    distances = np.zeros(len(X_scaled))  # اگر هیچ خوشه بزرگی نبود

# ------------------- ساخت model_value (0 تا 100) -------------------
anomaly_score = np.zeros(len(df))

for i, label in enumerate(labels):
    if label in large_cluster_labels:
        score = 0  # کاملاً نرمال
    else:
        # پایه: فاصله زیاد = امتیاز بالا
        dist_score = min(distances[i] / (eps * 3), 1.0) * 70  # حداکثر 70 از فاصله
        
        if label == -1:
            cluster_penalty = 30  # نویز خیلی غیرعادی
        elif label in small_cluster_labels:
            size = label_counts[label]
            cluster_penalty = max(10, 30 - size)  # خوشه 1 نفره → 29، خوشه 14 نفره → 16
        else:
            cluster_penalty = 0
        
        score = dist_score + cluster_penalty
        score = min(score, 100)  # حداکثر 100
    
    anomaly_score[i] = round(score, 2)

# ------------------- ساخت دیتافریم نهایی -------------------
result_df = df.copy()

# ستون DateTime
if 'RecordDate' in result_df.columns and 'RecordTime' in result_df.columns:
    result_df['DateTime'] = pd.to_datetime(
        result_df['RecordDate'].astype(str) + ' ' + result_df['RecordTime'].astype(str),
        errors='coerce'
    )
else:
    result_df['DateTime'] = pd.NaT

# اضافه کردن ستون‌های مدل
result_df['model_name'] = 'DBSCAN_Anomaly_Detection'
result_df['model_result'] = np.where(anomaly_score == 0, 'Normal', 'Anomaly')
result_df['model_value'] = anomaly_score

# مرتب‌سازی ستون‌ها به ترتیب خواسته شده
desired_columns = [
    'id', 'AssetID_8341', 'AssetID_8342', 'AssetID_8343', 'AssetID_8344',
    'AssetID_8346', 'AssetID_9286', 'AssetID_9287',
    'unitID', 'model_name', 'model_result', 'model_value',
    'DateTime', 'RecordDate', 'RecordTime', 'TimeStamps', 'created_at', 'updated_at'
]

# فقط ستون‌های موجود را نگه دار
final_columns = [col for col in desired_columns if col in result_df.columns]
# بقیه ستون‌های سنسور رو هم اضافه کن اگر نبودن
for col in sensor_columns:
    if col not in final_columns and col in result_df.columns:
        final_columns.insert(final_columns.index('unitID') if 'unitID' in final_columns else 8, col)

result_df = result_df[final_columns]

# مرتب‌سازی بر اساس زمان
result_df = result_df.sort_values('DateTime').reset_index(drop=True)

# ------------------- ذخیره فایل کامل مدل -------------------
print("در حال ذخیره فایل کامل مدل...")
result_df.to_excel(output_full_model, index=False)

# ------------------- ذخیره فقط انومالی‌ها (مثل قبل) -------------------
anomaly_mask_final = anomaly_score > 0
anomalies_df = result_df[anomaly_mask_final].copy()
anomalies_df.to_excel(output_anomalies, index=False)

# ------------------- خلاصه نهایی -------------------
n_anomalies = len(anomalies_df)
print("\n" + "="*70)
print("                  نتایج نهایی مدل DBSCAN")
print("="*70)
print(f"کل رکوردها            : {len(result_df):,}")
print(f"داده‌های عادی (Normal) : {len(result_df) - n_anomalies:,}")
print(f"داده‌های غیرعادی (Anomaly): {n_anomalies:,} ({n_anomalies/len(result_df)*100:.2f}%)")
print(f"بیشترین امتیاز انومالی: {anomaly_score.max():.1f}")
print(f"میانگین امتیاز انومالی در داده‌های غیرعادی: {anomaly_score[anomaly_score > 0].mean():.1f}")
print("="*70)
print(f"فایل کامل مدل ذخیره شد → {output_full_model}")
print(f"فایل فقط انومالی‌ها → {output_anomalies}")



در حال خواندن داده‌ها...
ستون‌های سنسور: 7 تا
اجرای DBSCAN...
محاسبه شدت انومالی (model_value)...
در حال ذخیره فایل کامل مدل...

                  نتایج نهایی مدل DBSCAN
کل رکوردها            : 10,927
داده‌های عادی (Normal) : 10,414
داده‌های غیرعادی (Anomaly): 513 (4.69%)
بیشترین امتیاز انومالی: 100.0
میانگین امتیاز انومالی در داده‌های غیرعادی: 70.5
فایل کامل مدل ذخیره شد → C:\BI\dbscan_model1.xlsx
فایل فقط انومالی‌ها → C:\BI\lube_oil_system_anomalies_g11.xlsx
